# Convolutional Networks Architectures

In [16]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [18]:
def evaluate_accuracy(data_iter, net, device):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum +=(net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [19]:
def train(net, train_iter, test_iter, optimizer, num_epochs, device):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
                print(f"Step {i}, time since epoch: {time.time() - start:.3f}. "
                      f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device), device)
        print(f"epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}" \
              f", test acc {test_acc:.3f}, time {time.time() - start:.1f} sec")    


## Dataset

In [20]:
BATCH_SIZE = 256
transforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('./datas', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.MNIST('./datas', train=False, transform=transforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## AlexNet

In [21]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [22]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
          Flatten-14                 [-

In [23]:
lr, num_epochs = 0.001, 1
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, optimizer, num_epochs, device)

Step 0, time since epoch: 11.125. Train acc: 0.082. Train Loss: 2.303
Step 10, time since epoch: 122.948. Train acc: 0.108. Train Loss: 2.337
Step 20, time since epoch: 230.337. Train acc: 0.212. Train Loss: 2.240
Step 30, time since epoch: 337.195. Train acc: 0.268. Train Loss: 2.080
Step 40, time since epoch: 444.090. Train acc: 0.375. Train Loss: 1.812
Step 50, time since epoch: 551.769. Train acc: 0.454. Train Loss: 1.590
Step 60, time since epoch: 658.362. Train acc: 0.515. Train Loss: 1.418
Step 70, time since epoch: 766.390. Train acc: 0.564. Train Loss: 1.279
Step 80, time since epoch: 873.164. Train acc: 0.606. Train Loss: 1.158
Step 90, time since epoch: 980.556. Train acc: 0.641. Train Loss: 1.058
Step 100, time since epoch: 1088.046. Train acc: 0.670. Train Loss: 0.975
Step 110, time since epoch: 1196.009. Train acc: 0.695. Train Loss: 0.903
Step 120, time since epoch: 1303.899. Train acc: 0.715. Train Loss: 0.845
Step 130, time since epoch: 1412.531. Train acc: 0.733. Trai